# 천지인 한글 입력 처리
## *202001551 정봉수*

In [3]:
# 초성: 19자
uni_choSung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
               'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성: 21자
uni_joongSung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 
                 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성: 28자(맨 앞의 공백 문자는 받침 없음)
uni_jongSung = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 
                 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
                 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

천지인 방식으로 모음 입력을 처리하기 위해 key와 모음 관계를 dict 타입으로 선언합니다.

In [4]:
vowel_map = {
    '1': 'ㅣ',  
    '12': 'ㅏ', '121': 'ㅐ', '122': 'ㅑ', '1221': 'ㅒ',  
    '21': 'ㅓ', '211': 'ㅔ',  
    '2': 'ㆍ', 
    '22' : 'ㆍㆍ', # 'ㆍㆍ'는 표준 천지인 조합은 아니나, 'ㅕ', 'ㅖ', 'ㅛ' 등의 모음을 만들기 위한 중간 과정 또는 내부 상태 표현을 위해 사용함.
                  # 예를 들어 'ㅕ'(221)를 입력하기 위해선 'ㆍ'(2) -> 'ㆍㆍ'(22) -> 'ㅕ'(221) 순으로 내부적으로 처리될 수 있음.
    '221': 'ㅕ', '2211': 'ㅖ', '223': 'ㅛ',  
    '23': 'ㅗ', '231': 'ㅚ', '2312': 'ㅘ', '23121': 'ㅙ', 
    '3': 'ㅡ', 
    '31': 'ㅢ', 
    '32': 'ㅜ', '321': 'ㅟ', '322': 'ㅠ',  
    '3221': 'ㅝ', '32211': 'ㅞ', 
}
#---fix---
#22 // 'ㆍ' + 'ㆍ' = 'ㆍㆍ' # 223 , 221 등과 같이 22가 모음 디셔너리에 있어야 다음 상태 판단 가능하여 추가 
#231 //'ㅗ' + 'ㅣ' = 'ㅚ'
#3 // 'ㅡ' 


In [5]:
# 202001551 정봉수 - last update (2025.05.12)

# 초성, 중성, 종성 리스트는 사전에 교수님께서 정의한 것을 사용하였습니더(uni_choSung, uni_joongSung, uni_jongSung)


# 천지인 키패드 자음 및 기본 모음 키 매핑:
# - 모음 키('1','2','3')는 해당하는 기본 모음 문자를 직접 값으로 가짐. 이들은 모음 조합의 시작점이 됨.
# - 자음 키('4'~'0')는 해당 키를 반복적으로 눌렀을 때 순환(토글)되는 자음들의 리스트를 값으로 가짐.
key_map = {
    '1': 'ㅣ', '2': 'ㆍ', '3': 'ㅡ',  # 모음 키와 해당 기본 모음 문자. vowel_map의 단일 키에 해당함.
    '4':['ㄱ','ㅋ','ㄲ'],          # '4' 키: 'ㄱ' -> 'ㅋ' -> 'ㄲ' 순으로 토글됨. 다시 누르면 'ㄱ'으로 돌아감.
    '5':['ㄴ','ㄹ'],              # '5' 키: 'ㄴ' -> 'ㄹ' 순으로 토글됨.
    '6':['ㄷ','ㅌ','ㄸ'],          # '6' 키: 'ㄷ' -> 'ㅌ' -> 'ㄸ' 순으로 토글됨.
    '7':['ㅂ','ㅍ','ㅃ'],          # '7' 키: 'ㅂ' -> 'ㅍ' -> 'ㅃ' 순으로 토글됨.
    '8':['ㅅ','ㅎ','ㅆ'],          # '8' 키: 'ㅅ' -> 'ㅎ' -> 'ㅆ' 순으로 토글됨.
    '9':['ㅈ','ㅊ','ㅉ'],          # '9' 키: 'ㅈ' -> 'ㅊ' -> 'ㅉ' 순으로 토글됨.
    '0':['ㅇ','ㅁ']               # '0' 키: 'ㅇ' -> 'ㅁ' 순으로 토글됨.
}

# 겹받침(종성) 조합 규칙을 정의함.
# 키는 (첫 번째 자음, 두 번째 자음)의 튜플 형태이고, 값은 조합된 겹받침 문자임.
# 이 딕셔너리는 두 개의 단일 자음으로 겹받침을 형성할 수 있는지, 또는 형성된 겹받침이 무엇인지 판단하는 데 사용됨.
COMPOUND_JONG = {
    ('ㄱ', 'ㅅ'): 'ㄳ', ('ㄴ', 'ㅈ'): 'ㄵ', ('ㄴ', 'ㅎ'): 'ㄶ',
    ('ㄹ', 'ㄱ'): 'ㄺ', ('ㄹ', 'ㅁ'): 'ㄻ', ('ㄹ', 'ㅂ'): 'ㄼ',
    ('ㄹ', 'ㅅ'): 'ㄽ', ('ㄹ', 'ㅌ'): 'ㄾ', ('ㄹ', 'ㅍ'): 'ㄿ',
    ('ㄹ', 'ㅎ'): 'ㅀ', ('ㅂ', 'ㅅ'): 'ㅄ'
}

# 겹받침 분리 규칙: 모음 앞에서 겹받침이 어떻게 분리되는지 정의
# Key: 겹받침 문자
# Value: (남는 종성, 다음 초성으로 갈 자음) 튜플. 남는 종성이 없으면 None 또는 빈 문자열.
COMPOUND_JONG_SPLIT_RULES = {
    'ㄳ': ('ㄱ', 'ㅅ'),
    'ㄵ': ('ㄴ', 'ㅈ'),
    'ㄶ': ('ㄴ', 'ㅎ'),
    'ㄺ': ('ㄹ', 'ㄱ'), # 예: '닭' + 'ㅣ' -> '달' + '기'
    'ㄻ': ('ㄹ', 'ㅁ'),
    'ㄼ': ('ㄹ', 'ㅂ'), # 예: '넓' + 'ㅣ' -> '널' + '비' 
    'ㄽ': ('ㄹ', 'ㅅ'),
    'ㄾ': ('ㄹ', 'ㅌ'),
    'ㄿ': ('ㄹ', 'ㅍ'),
    'ㅀ': ('ㄹ', 'ㅎ'),
    'ㅄ': ('ㅂ', 'ㅅ')  # 예: '없' + 'ㅣ' -> '업' + '시'
}

def get_jamo_index(jamo_list, jamo_char):
    '''
    주어진 자모 리스트(예: uni_choSung)에서 특정 자모 문자의 인덱스를 반환함.
    한글 음절 조합 시 각 자모가 해당 리스트에서 몇 번째인지 알아야 유니코드 계산이 가능함.

    Args:
        jamo_list (list): 자모 문자들로 이루어진 리스트 (예: uni_choSung, uni_joongSung, uni_jongSung).
        jamo_char (str): 인덱스를 찾고자 하는 자모 문자.

    Returns:
        int: `jamo_list` 내에서 `jamo_char`의 인덱스. 문자가 리스트에 없으면 -1을 반환함.
    '''
    try:
        return jamo_list.index(jamo_char) # 리스트에 자모가 있으면 해당 인덱스 반환함.
    except ValueError: # 리스트에 자모가 없을 경우 ValueError 발생함.
        return -1 # 오류 대신 -1을 반환하여 후속 처리에서 유효하지 않음을 알림.

def make_syllable_from_indices(cho_idx, joong_idx, jong_idx):
    '''
    초성, 중성, 종성의 인덱스 값을 사용하여 완전한 한글 음절의 유니코드 문자를 생성함.
    각 인덱스는 해당 자모가 uni_choSung, uni_joongSung, uni_jongSung 리스트에서 차지하는 순서임.
    유니코드 한글 음절은 '가'(0xAC00)를 기준으로 특정 계산식에 따라 코드가 부여됨.

    Args:
        cho_idx (int): 초성 자모의 uni_choSung 리스트 내 인덱스.
        joong_idx (int): 중성 자모의 uni_joongSung 리스트 내 인덱스.
        jong_idx (int): 종성 자모의 uni_jongSung 리스트 내 인덱스 (받침 없으면 공백(' ')의 인덱스).

    Returns:
        str or None: 생성된 한글 음절 문자. 초성 또는 중성 인덱스가 유효하지 않으면 None을 반환함.
    '''
    # 초성이나 중성 인덱스가 -1이면 (즉, 유효하지 않은 자모이거나 자모가 없는 초기 상태 등) 음절을 만들 수 없음.
    if cho_idx == -1 or joong_idx == -1 :
        return None # 음절 생성 불가를 나타내는 None 반환함.
    # 한글 음절 유니코드 계산 공식: (초성 인덱스 * 21 * 28) + (중성 인덱스 * 28) + 종성 인덱스 + 0xAC00('가')
    # 21은 중성의 개수, 28은 종성의 개수(공백 포함)를 의미함.
    return chr(0xAC00 + (cho_idx * 21 * 28) + (joong_idx * 28) + jong_idx)


class CheonjiinHangulAutomata:
    def __init__(self):
        # 현재 오토마타의 상태를 나타냄. 각 상태는 한글 입력 과정의 특정 단계를 의미함.
        # S0: 초기 상태 (입력 대기 또는 한 음절 완성 직후)
        # S1: 초성만 입력된 상태
        # S2: 초성 + 중성까지 입력된 상태
        # S3: 초성 + 중성 + 종성까지 입력된 상태 (단일 종성 또는 겹받침 형성/토글 중인 상태 포함)
        self.current_state_enum = "S0"
        self.output_buffer = []         # 완성된 한글 음절이나 단독 자모를 저장하는 리스트. 최종 출력은 이 리스트를 합쳐서 만듦.
        self.cho = ''                   # 현재 조합 중인 음절의 초성 자모를 저장함.
        self.jung = ''                  # 현재 조합 중인 음절의 중성 자모를 저장함.
        self.jong = ''                  # 현재 조합 중인 음절의 종성 자모를 저장함.
        self.vowel_input_buffer = ""    # 모음 키('1','2','3') 입력 시퀀스를 저장하는 버퍼.
        self.consonant_key_pressed = None # 마지막으로 눌린 자음 키(예: '4', '5')를 저장함. 자음 토글 처리에 사용됨.
        self.consonant_press_count = 0  # 마지막으로 눌린 자음 키의 연속 입력 횟수를 저장함.
        self.last_key_was_space = False # 직전 입력 키가 공백이었는지 여부를 저장함.
        # Lookahead 기능을 위해 전체 입력 시퀀스를 저장할 변수. process_input_sequence 호출 시 설정됨.
        self.current_input_sequence = ""
        
    def get_output(self):
        '''
        현재까지 `output_buffer`에 저장된 모든 완성된 글자들을 하나의 문자열로 합쳐 반환함.
        '''
        return "".join(self.output_buffer)

    def _clear_syllable_buffers(self, clear_output_too=False):
        '''
        현재 조합 중인 음절과 관련된 내부 버퍼들을 초기화함.
        '''
        self.cho = ''
        self.jung = ''
        self.jong = ''
        self.vowel_input_buffer = ""
        self.consonant_key_pressed = None
        self.consonant_press_count = 0
        if clear_output_too:
            self.output_buffer = []

    def _commit_syllable(self, new_cho_for_next_syllable=None, new_jung_for_next_syllable=None):
        '''
        현재 자모들을 바탕으로 한글 음절을 완성하고 `output_buffer`에 추가함.
        '''
        syllable_made = False
        if self.cho and self.jung: # 초성과 중성이 모두 있어야 완전한 음절.
            cho_idx = get_jamo_index(uni_choSung, self.cho)
            joong_idx = get_jamo_index(uni_joongSung, self.jung)
            jong_char_for_lookup = self.jong if self.jong else ' ' # 종성 없으면 공백으로 처리.
            jong_idx = get_jamo_index(uni_jongSung, jong_char_for_lookup)
            syllable = make_syllable_from_indices(cho_idx, joong_idx, jong_idx)
            if syllable:
                self.output_buffer.append(syllable)
                syllable_made = True
        elif self.cho: # 초성만 있는 경우 (단독 자음).
            self.output_buffer.append(self.cho)
            syllable_made = True
        elif self.jung: # 중성만 있는 경우 (단독 모음).
             # 초성, 종성 없이 중성만으로 이루어진 경우에만 단독 모음으로 커밋.
            if not self.cho and not self.jong:
                 self.output_buffer.append(self.jung)
                 syllable_made = True

        self._clear_syllable_buffers() # 커밋 후 다음 입력을 위해 버퍼 초기화.

        # 다음 음절이 바로 초성+중성 또는 중성만으로 시작되는 경우 처리.
        if new_cho_for_next_syllable and new_jung_for_next_syllable:
            self.cho = new_cho_for_next_syllable
            self.jung = new_jung_for_next_syllable
            self.current_state_enum = "S2"
        elif new_jung_for_next_syllable: 
            self.jung = new_jung_for_next_syllable
            self.current_state_enum = "S2"
        else: # 그 외에는 초기 상태 S0로.
            self.current_state_enum = "S0"
            
        return syllable_made

    # process_key 메소드는 이제 현재 처리 중인 키, 해당 키의 전체 시퀀스 내 인덱스,
    # 그리고 전체 시퀀스의 길이를 인자로 받습니다.
    # 반환값은 이 호출로 인해 "소모된" 입력 키의 개수입니다 (lookahead으로 여러 키를 한 번에 처리한 경우 > 1).
    def process_key(self, key, current_key_idx, full_sequence_len):
        '''
        단일 키 하나를 처리. 현재 오토마타 상태와 키 종류에 따라 내부 상태 변경 및 음절 조합.
        Args:
            key (str): 입력된 단일 키 문자.
            current_key_idx (int): 전체 입력 시퀀스에서 현재 'key'의 인덱스.
            full_sequence_len (int): 전체 입력 시퀀스의 길이.
        Returns:
            int: 이번 키 처리로 인해 소모된 입력 문자열의 길이 (기본 1).
        '''
        keys_consumed_this_call = 1 # 기본적으로 현재 키 1개만 소모된 것으로 시작.
        current_key_is_space = (key == ' ')

        # 1. 공백 키(' ') 처리 로직
        if current_key_is_space:
            # 현재 조합 중인 글자(초/중/종성 중 하나라도 존재)가 있다면, 이를 완성하여 출력 버퍼에 넣음.
            if self.cho or self.jung or self.jong:
                self._commit_syllable()

            # 이전 입력도 공백이었다면 (즉, 연속된 공백 입력), 실제 공백 문자(' ')를 출력 버퍼에 추가함.
            # 천지인에서는 첫 번째 공백은 글자 완결의 의미로, 두 번째 연속된 공백부터 실제 공백으로 입력되는 경우가 많음.
            if self.last_key_was_space:
                self.output_buffer.append(' ')

            self.current_state_enum = "S0" # 공백 입력 후에는 항상 초기 상태로 돌아감.
            self.last_key_was_space = True # 마지막 키가 공백이었음을 기록함.
            return keys_consumed_this_call # 공백키는 1개만 소모.

        # 공백이 아닌 다른 키가 입력된 경우, `last_key_was_space`를 False로 설정함.
        self.last_key_was_space = False

        # 2. 현재 오토마타 상태(`self.current_state_enum`)에 따른 분기 처리
        # 상태 S0: 초기 상태 (아무것도 입력되지 않았거나, 한 글자가 완성된 직후의 상태)
        if self.current_state_enum == "S0":
            '''
            [S0: 초기 상태] 처리 로직:
            - 입력 대기 중이거나 한 음절이 막 완성된 상태임.
            - 자음 키 입력: 새 음절의 초성으로 처리하고 S1 상태로 전이함.
            - 기본 모음 키('1','2','3') 입력: 해당 모음을 중성으로 설정하고 S2 상태로 전이함 (초성은 비어 있음).
            '''
            if key in key_map:
                # S0에서 새 글자 시작 전, 만약 이전 _commit_syllable에서 처리되지 못한
                # 단독 자/모음(예: 모음 조합 실패 후 남은 단일 모음)이 버퍼에 남아있을 수 있으므로,
                # 이를 마저 커밋할 수 있도록 처리. (일반적인 음절 완성 후 S0 진입 시에는 버퍼가 비어있음)
                if self.cho or self.jung: 
                     self._commit_syllable()

                self._clear_syllable_buffers() # 새 글자 시작 전 항상 버퍼 초기화.
                value_from_key_map = key_map[key]

                if isinstance(value_from_key_map, list): # 자음 키 입력 ('4' ~ '0')
                    self.consonant_key_pressed = key
                    self.consonant_press_count = 1
                    self.cho = value_from_key_map[0] # 자음 리스트의 첫 번째 자음을 초성으로 설정
                    self.current_state_enum = "S1" # 초성 입력 상태로 전이
                else: # 기본 모음 키('1','2','3') 입력
                    self.vowel_input_buffer = key # 모음 키 자체를 버퍼에 저장 (예: '1')
                    current_vowel_candidate = vowel_map.get(self.vowel_input_buffer) # vowel_map에서 해당 키의 모음 조회 (예: 'ㅣ')
                    if current_vowel_candidate:
                        self.jung = current_vowel_candidate # 조회된 모음을 현재 음절의 중성으로 설정
                    else:
                        # 이 경우는 '1','2','3'이 vowel_map에 단일 키로 정의되어 있으므로 발생하지 않아야 함.
                        # (key_map[key]의 값이 vowel_map의 키와 동일하게 설정되어 있기 때문)
                        self.jung = '' # 방어적으로 빈 문자열 할당
                    self.current_state_enum = "S2" # (초성 없이) 중성 입력/조합 상태로 전이
            else:
                pass # 유효하지 않은 키 입력은 무시함. (key_map에 없는 키)

        # 상태 S1: 초성만 입력된 상태
        elif self.current_state_enum == "S1":
            '''
            [S1: 초성 입력 상태] 처리 로직:
            - 현재 음절의 초성(`self.cho`)이 설정된 상태임.
            - 같은 자음 키 반복 입력: 초성을 토글함 (예: 'ㄱ' -> 'ㅋ'). 상태 S1 유지.
            - 다른 자음 키 입력: 현재 초성을 단독 자음으로 커밋하고, 새 자음을 새 초성으로. 상태 S1 유지.
            - 기본 모음 키('1','2','3') 입력: 현재 초성과 결합할 중성으로 처리하고 S2 상태로 전이함.
            '''
            if key == self.consonant_key_pressed: # 이전에 눌렀던 자음 키와 같은 키가 입력된 경우 (초성 토글)
                self.consonant_press_count += 1
                options = key_map[key] # 해당 키의 자음 옵션 리스트
                self.cho = options[(self.consonant_press_count - 1) % len(options)] # 다음 자음으로 초성 변경
            elif key in key_map and isinstance(key_map[key], list): # 다른 자음 키가 입력된 경우
                self.output_buffer.append(self.cho) # 현재까지 입력된 초성을 단독 자음으로 출력 버퍼에 추가 (커밋)
                self._clear_syllable_buffers() # 새 초성 입력을 위해 버퍼 초기화
                self.consonant_key_pressed = key # 새로 입력된 자음 키 정보 저장
                self.consonant_press_count = 1
                self.cho = key_map[key][0] # 새 자음 리스트의 첫 번째 자음을 초성으로 설정
                # self.current_state_enum = "S1" # 상태는 S1으로 유지됨
            elif key in key_map and isinstance(key_map[key], str): # 기본 모음 키 ('1','2','3')가 입력된 경우
                self.vowel_input_buffer = key # 입력된 모음 키를 모음 조합 버퍼에 저장
                current_vowel_candidate = vowel_map.get(self.vowel_input_buffer) # vowel_map에서 해당 모음 조회
                if current_vowel_candidate:
                    self.jung = current_vowel_candidate # 조회된 모음을 현재 음절의 중성으로 설정
                else:
                    # '1','2','3'은 key_map과 vowel_map에 단일 키로 정의되어 있으므로,
                    # current_vowel_candidate가 None이 되는 경우는 이론적으로 발생하지 않아야 함.
                    self.jung = '' # 방어적으로 빈 문자열 할당
                self.current_state_enum = "S2" # 초성+중성 입력 상태로 전이
            else:
                pass # 유효하지 않은 키 입력은 무시함.

        # 상태 S2: 초성 + 중성까지 입력된 상태
        elif self.current_state_enum == "S2":
            '''
            [S2: 중성 입력 상태] 처리 로직:
            - 현재 음절의 초성(`self.cho`)과 중성(`self.jung`)이 설정된 상태임.
            - 기본 모음 키('1','2','3') 추가 입력: 현재 중성과 조합하여 복합 모음을 만들거나,
              더 이상 조합이 불가능하면 현재 음절(초성+중성)을 커밋하고 새 단독 모음으로 시작. 상태 S2 유지.
            - 자음 키 입력: 현재 음절의 종성으로 처리하고 S3 상태로 전이함.
            '''
            if key in key_map and isinstance(key_map[key], str): # 또 다른 기본 모음 키 ('1','2','3')가 입력된 경우 (모음 조합 시도)
                temp_vowel_buffer = self.vowel_input_buffer + key # 기존 모음 입력 버퍼에 새 모음 키 추가 (예: '1' + '2' -> '12')
                new_vowel = vowel_map.get(temp_vowel_buffer) # 조합된 모음 후보를 vowel_map에서 조회
                
                if new_vowel: # 모음 조합이 가능한 경우 (vowel_map에 해당 시퀀스가 있는 경우)
                    self.vowel_input_buffer = temp_vowel_buffer # 모음 입력 버퍼 업데이트
                    if new_vowel == 'ㆍㆍ' and self.jung == 'ㆍ': # 'ㆍ' 다음에 'ㆍ'가 와서 'ㆍㆍ'가 되는 특정 케이스 처리
                        self.jung = new_vowel # 중성을 'ㆍㆍ'로 업데이트
                    elif new_vowel != 'ㆍㆍ': # 'ㆍㆍ'가 아닌 다른 유효한 조합 모음인 경우
                         self.jung = new_vowel # 중성을 새 조합 모음으로 업데이트
                    # self.current_state_enum = "S2" # 상태는 S2로 유지됨 (더 많은 모음 조합 또는 자음 입력 대기)
                else: # 모음 조합 실패 (더 이상 조합할 수 없는 시퀀스인 경우)
                    self._commit_syllable() # 현재까지 조합된 음절(초성+중성 또는 중성만)을 커밋 (출력 버퍼에 추가)
                    # 새 모음으로 새 음절(중성만) 시작.
                    self.vowel_input_buffer = key # 새 모음 키로 버퍼 초기화
                    current_vowel_candidate = vowel_map.get(self.vowel_input_buffer)
                    if current_vowel_candidate:
                        self.jung = current_vowel_candidate
                    else:
                        self.jung = ''
                    self.current_state_enum = "S2" # 새 음절이 모음(중성)만으로 시작
            elif key in key_map and isinstance(key_map[key], list): # 자음 키 ('4' ~ '0')가 입력된 경우 (종성 입력 시작)
                self.vowel_input_buffer = "" # 모음 조합 버퍼는 더 이상 필요 없으므로 초기화
                self.consonant_key_pressed = key # 새로 입력된 자음 키 정보 저장
                self.consonant_press_count = 1
                self.jong = key_map[key][0] # 해당 자음 리스트의 첫 번째 자음을 종성으로 설정
                self.current_state_enum = "S3" # 초성+중성+종성 입력 상태로 전이
            else:
                pass # 유효하지 않은 키 입력은 무시함.

        # 상태 S3: 초성 + 중성 + 종성까지 입력된 상태
        elif self.current_state_enum == "S3":
            '''
            [S3: 종성 입력 상태] 처리 로직 (단일 종성 및 겹받침 처리 포함):
            - 현재 음절의 초성, 중성, 종성(`self.jong`)이 설정된 상태임.
            - 같은 종성 자음 키 반복 입력: 종성 토글 또는 겹받침 내 토글/변경.
            - 다른 자음 키 입력: 미래 입력을 참조(lookahead)하여 최적의 겹받침을 형성하거나, 실패 시 현재 음절 커밋 후 새 초성으로 시작.
            - 기본 모음 키('1','2','3') 입력: 겹받침 분리, 연음 또는 현재 음절 커밋 후 새 모음으로 새 음절 시작.
            '''
            if key == self.consonant_key_pressed: # 이전에 눌렀던 자음 키와 같은 키가 입력된 경우 (종성 토글 또는 겹받침 내 토글)
                self.consonant_press_count += 1
                options = key_map[key] # 해당 키의 자음 옵션 리스트
                next_toggled_simple_consonant = options[(self.consonant_press_count - 1) % len(options)] # 토글될 단일 자음 후보

                original_jong = self.jong # 현재 종성 저장
                is_original_jong_compound = False # 현재 종성이 겹받침인지 여부
                first_part_of_compound = None # 겹받침일 경우 첫 번째 구성 자음
                
                for c1_iter, c2_iter in COMPOUND_JONG.keys(): # 모든 겹받침 정의 순회
                    if COMPOUND_JONG[(c1_iter, c2_iter)] == original_jong: # 현재 종성이 겹받침 리스트에 있다면
                        # 겹받침의 두 번째 요소(c2_iter)가 현재 눌린 키(key)의 옵션들(options)에 속하는지 확인.
                        # 이 조건이 있어야 'ㄺ' 상태에서 'ㄱ'계열 키('4')를 눌렀을 때만 'ㄹ'을 첫 부분으로 하는 토글을 시도함.
                        if c2_iter in options:
                            is_original_jong_compound = True
                            first_part_of_compound = c1_iter # 겹받침의 첫번째 부분을 저장
                            break
                
                if is_original_jong_compound: # 현재 종성이 겹받침이고, 현재 키가 그 일부를 토글 가능할 때
                    # 겹받침의 첫 부분(first_part_of_compound)과 새로 토글된 단일 자음(next_toggled_simple_consonant)으로 새 겹받침 시도
                    if (first_part_of_compound, next_toggled_simple_consonant) in COMPOUND_JONG:
                        self.jong = COMPOUND_JONG[(first_part_of_compound, next_toggled_simple_consonant)] # 새 겹받침으로 업데이트
                    else: 
                        # 새 겹받침 조합이 불가능하면, 원래 겹받침을 유지하고 press_count를 원복 (더 이상 토글 안됨을 의미)
                        self.consonant_press_count -= 1 
                        self.jong = original_jong # 변화 없음
                else: # 현재 종성이 단일 자음이거나, 겹받침이지만 현재 키로 토글할 수 없는 타입일 경우
                    self.jong = next_toggled_simple_consonant # 단순 토글된 자음으로 종성 업데이트
                # self.current_state_enum = "S3" # 상태는 S3로 유지됨
                # keys_consumed_this_call는 1로 유지됨.

            elif key in key_map and isinstance(key_map[key], list): # S3에서 "다른 자음 키"가 입력된 경우
                # 이 부분은 미래의 키 입력을 참조(lookahead)하여 겹받침을 결정하는 로직입니다.
                # 1. 미래 입력을 포함하여 최적의 겹받침을 찾습니다.
                # 2. 찾지 못하면, 현재 키의 첫 옵션만으로 단순 겹받침을 시도합니다.
                # 3. 그것도 실패하면, 현재 음절을 커밋하고 새 음절을 시작합니다.
                # !! 실제로는 실시간 입력상황임으로 미래 정보를 보는것이 불가능하여, 시간지연 방식 구현이 올바르나, 현재 상황에서는 해당 방식을 사용하였습니다. !!
                
                current_syllable_jong = self.jong 
                options_for_new_key = key_map[key]
                
                # 찾은 최적의 겹받침 정보를 저장할 변수:
                # {'jong': 생성될 겹받침 문자, 
                #  'consumed_keys': 이 겹받침 형성에 소모될 총 키의 수 (현재 키 + lookahead된 키),
                #  'new_count': 새로 눌린 자음 키의 consonant_press_count 값}
                best_compound_formation = None 

                # 겹받침 시도는 현재 종성이 단일 자음일 때만 유효.
                if current_syllable_jong and len(current_syllable_jong) == 1 and \
                   (current_syllable_jong in uni_choSung or current_syllable_jong in uni_jongSung[1:]):

                    # 1단계: Lookahead를 사용하여 미래의 토글까지 고려한 겹받침 탐색
                    # 가장 많은 후속 키 입력을 정확히 예측하여 형성되는 겹받침을 우선적으로 찾습니다.
                    temp_best_formation_candidate = None
                    max_consumed_for_candidate = 0 # 현재까지 찾은 최선책이 소모하는 키의 수

                    for i, ki_option in enumerate(options_for_new_key): # 새 자음키의 각 옵션(ki_option)에 대해
                        prospective_compound = COMPOUND_JONG.get((current_syllable_jong, ki_option))
                        if prospective_compound: # (현재 종성 + ki_option)이 겹받침을 형성할 수 있다면
                            needed_toggles = i # 이 ki_option을 만들기 위해 필요한 토글 횟수 (0이면 첫 옵션)
                            
                            can_form_with_lookahead = True # 이 옵션으로 겹받침 형성이 최종적으로 가능한지 여부
                            if needed_toggles > 0: # 첫 번째 옵션이 아니라면, 실제 후속 입력이 있는지 확인해야 함
                                if current_key_idx + needed_toggles >= full_sequence_len: # 필요한 만큼의 후속 키가 문자열에 없음
                                    can_form_with_lookahead = False
                                else:
                                    # 필요한 토글 수만큼 다음 키들이 현재 입력된 자음 키(key)와 동일한지 확인
                                    for j in range(needed_toggles):
                                        if self.current_input_sequence[current_key_idx + 1 + j] != key:
                                            can_form_with_lookahead = False # 후속 키 불일치
                                            break
                            
                            if can_form_with_lookahead: # Lookahead 성공 또는 필요 없음 (첫 옵션)
                                num_keys_for_this_option = 1 + needed_toggles # 현재 키 + 예측된 토글 키 수
                                # 더 많은 키를 소모하는 (더 확실한 예측의) 겹받침을 최선으로 간주
                                if num_keys_for_this_option > max_consumed_for_candidate:
                                    max_consumed_for_candidate = num_keys_for_this_option
                                    temp_best_formation_candidate = {
                                        'jong': prospective_compound,
                                        'consumed_keys': num_keys_for_this_option,
                                        'new_count': i + 1 
                                    }
                    if temp_best_formation_candidate: # Lookahead를 통해 유효한 겹받침을 찾았다면
                        best_compound_formation = temp_best_formation_candidate
                
                # 위에서 찾은 최적의 겹받침 정보(best_compound_formation)를 바탕으로 최종 처리
                if best_compound_formation:
                    # Lookahead를 통해 겹받침 형성 결정
                    self.jong = best_compound_formation['jong']
                    self.consonant_key_pressed = key # 마지막 누른 자음키 정보 업데이트
                    self.consonant_press_count = best_compound_formation['new_count'] # 해당 자음키의 옵션 count
                    keys_consumed_this_call = best_compound_formation['consumed_keys'] # 소모된 총 키 수 업데이트
                    # 상태는 S3로 유지
                else:
                    # Lookahead를 통한 겹받침 형성 실패 (또는 현재 종성이 단일자음이 아니어서 시도 안함).
                    # 2단계: 단순 겹받침 시도 (현재 입력된 자음 키의 첫 번째 옵션과, lookahead 없이)
                    first_option_char = options_for_new_key[0]
                    simple_compound_candidate = None
                    # 이 단순 겹받침 시도 역시 현재 종성이 단일 자음일 때만 유효
                    if current_syllable_jong and len(current_syllable_jong) == 1 and \
                       (current_syllable_jong in uni_choSung or current_syllable_jong in uni_jongSung[1:]):
                        simple_compound_candidate = COMPOUND_JONG.get((current_syllable_jong, first_option_char))

                    if simple_compound_candidate:
                        # 단순 겹받침 성공 (예: '갃' = ㄱ+ㅏ+ㄱ 에 'ㅅ'키 입력시 바로 ㄱ+ㅏ+ㄳ)
                        self.jong = simple_compound_candidate
                        self.consonant_key_pressed = key
                        self.consonant_press_count = 1 # 첫 옵션이므로 count는 1
                        keys_consumed_this_call = 1 # 현재 키 하나만 소모
                        # 상태는 S3로 유지
                    else:
                        # 3단계: 모든 종류의 겹받침 형성 실패 -> 현재 음절 커밋하고 새 글자 시작
                        # (예: '간ㅅ' = ㄱ+ㅏ+ㄴ 에 'ㅅ'키 입력시 'ㄴ'+'ㅅ' 겹받침 없으므로 분리)
                        self._commit_syllable() 
                        
                        self.cho = first_option_char # 새 자음키의 첫 옵션을 새 음절의 초성으로
                        self.consonant_key_pressed = key
                        self.consonant_press_count = 1
                        self.current_state_enum = "S1" # 새 음절은 초성만 입력된 상태로 시작
                        keys_consumed_this_call = 1 # 현재 키 하나만 소모
            
            elif key in key_map and isinstance(key_map[key], str): # S3에서 모음 키 입력
                split_rule = COMPOUND_JONG_SPLIT_RULES.get(self.jong) # 현재 종성이 겹받침이고 분리 규칙이 있는지 확인
                
                if split_rule: # 겹받침 분리 규칙이 있는 경우
                    remaining_jong, next_cho = split_rule # (남는 종성, 다음 음절의 초성이 될 자음)
                    self.jong = remaining_jong if remaining_jong else '' # 현재 음절의 종성을 남는 종성으로 업데이트 (없으면 빈 문자열)
                    
                    self.vowel_input_buffer = key # 새 모음 키 저장 (다음 음절의 중성을 위함)
                    # 현재 음절(남은 종성으로) 커밋, 분리된 자음(next_cho)을 다음 음절 초성으로, 새 모음을 다음 음절 중성으로 설정
                    self._commit_syllable(new_cho_for_next_syllable=next_cho, new_jung_for_next_syllable=vowel_map.get(key))
                    # _commit_syllable 내부에서 current_state_enum이 S2로 설정됨 (다음 음절은 초성+중성으로 시작)
                else: # 단일 종성이거나, 겹받침이지만 분리 규칙이 없는 경우
                    original_jong_char = self.jong # 현재 종성 문자 저장
                    # 현재 종성이 존재하고, 단일 문자이며, 유효한 초성 문자인지 확인 (' ' 제외)
                    if len(original_jong_char) == 1 and original_jong_char in uni_choSung and original_jong_char != ' ':
                        # 종성이 다음 글자의 초성으로 넘어갈 수 있는 경우 (예: '각'+'ㅣ' -> '가'+'기')
                        self.jong = '' # 현재 음절에서 종성을 제거 (초성+중성만 남김)
                        self._commit_syllable(new_cho_for_next_syllable=original_jong_char, # 원래 종성을 다음 음절의 초성으로
                                              new_jung_for_next_syllable=vowel_map.get(key))  # 새 모음을 다음 음절의 중성으로
                        self.vowel_input_buffer = key # 다음 음절의 중성을 위한 모음 키 저장
                        # _commit_syllable 내부에서 current_state_enum이 S2로 설정됨
                    else:
                        # 종성을 다음 글자의 초성으로 넘길 수 없는 경우
                        # (예: 종성이 없거나, 겹받침이거나(분리규칙없는), 초성으로 사용 못하는 문자이거나 등)
                        # 현재 음절(초성+중성+종성)을 그대로 커밋하고, 새 모음으로 새 음절 시작.
                        self._commit_syllable() 
                        # 커밋 후 상태는 S0가 됨. 새 모음 입력을 새 음절의 중성으로 처리.
                        self.vowel_input_buffer = key 
                        current_vowel_candidate = vowel_map.get(self.vowel_input_buffer)
                        if current_vowel_candidate:
                            self.jung = current_vowel_candidate
                        else:
                            # '1','2','3'은 vowel_map에 단일 키로 존재하므로 이 경우는 발생하지 않아야 함.
                            self.jung = '' # 방어적 할당
                        self.current_state_enum = "S2" # 새 음절이 (초성 없이) 중성만으로 시작
            else: # 유효하지 않은 키 입력은 무시함.
                pass # keys_consumed_this_call은 1로 유지.
        
        return keys_consumed_this_call

    def process_input_sequence(self, input_sequence_str):
        '''
        전체 입력 키 시퀀스 문자열(`input_sequence_str`)을 한 글자씩 순차적으로 처리하는 메서드임.
        입력 처리 시작 전에 모든 내부 상태와 버퍼를 완전히 초기화함.
        각 키에 대해 `process_key` 메서드를 호출하여 개별 키 처리를 수행함.
        모든 키 처리가 끝난 후, 만약 아직 커밋되지 않은 조합 중인 음절이 남아있다면 마지막으로 커밋함.
        최종적으로 완성된 전체 한글 문자열을 반환함.

        Args:
            input_sequence_str (str): 처리할 전체 키 입력 시퀀스 문자열.

        Returns:
            str: 최종적으로 조합된 전체 한글 문자열.
        '''
        self._clear_syllable_buffers(clear_output_too=True) # 모든 버퍼와 상태 초기화 (출력 버퍼 포함)
        self.current_state_enum = "S0" # 초기 상태에서 시작
        self.last_key_was_space = False # 이전 키 공백 여부 초기화
        # 전체 입력 시퀀스를 클래스 멤버에 저장하여 process_key 내부의 lookahead 로직에서 참조할 수 있도록 함.
        self.current_input_sequence = input_sequence_str

        idx = 0 # 문자열 처리를 위한 현재 인덱스.
        sequence_len = len(input_sequence_str)
        while idx < sequence_len:
            char_key = input_sequence_str[idx] # 현재 처리할 키.
            
            # process_key는 현재 키, 현재 키의 인덱스, 전체 시퀀스 길이를 인자로 받고,
            # 이번 호출로 처리(소모)된 키의 개수를 반환함.
            consumed_count = self.process_key(char_key, idx, sequence_len)
            idx += consumed_count # 다음 처리할 키의 인덱스로 이동.

        # 모든 키 입력 처리 후, 아직 조합 중인 음절(초/중/종성 중 하나라도 존재)이 남아있다면
        if self.cho or self.jung or self.jong: 
            self._commit_syllable() # 마지막으로 커밋하여 완성

        return self.get_output() # 최종 조합된 전체 문자열 반환

In [7]:
    # --- 테스트 ---
    automata = CheonjiinHangulAutomata()

    '''uni_jongSung = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ',
                    'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ',
                    'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']'''
    '''key_map = {
        '4': ['ㄱ', 'ㅋ', 'ㄲ'],  
        '5': ['ㄴ', 'ㄹ'],    
        '6': ['ㄷ', 'ㅌ', 'ㄸ'],
        '7': ['ㅂ', 'ㅍ', 'ㅃ'],
        '8': ['ㅅ', 'ㅎ', 'ㅆ'],
        '9': ['ㅈ', 'ㅊ', 'ㅉ'],
        '0': ['ㅇ', 'ㅁ']
    }'''                 

    test_cases = {
        "4127": "갑",
        "44120": "캉",
        "412": "가",
        "8214": "석",
        "88823121":"쐐",
        "9315": "즨", # 'ㅈ' + '-' + 'ㅣ' + 'ㄴ' -> '즨'
        "41254": "간ㄱ", # 공백 없이는 이렇게 출력됨
        "4125412": "간가",
        "7771": "삐",
        "0210": "엉",
        "02188": "엏",
        "412 412": "가가", # 공백 하나는 커밋, 두 번째 공백부터 실제 공백 
        "0127 7124": "압박", # ㅇㅏㅂㅂㅏㄱ -> ㅇㅏㅂ (압) commit. 다음 ㅂ 초성 -> 박
        " ": "", # 공백 하나는 아무것도 출력 안 함
        "  ": " ", # 공백 두 개는 실제 공백 하나 출력
        "   ": "  ",# 공백 세 개는 실제 공백 두 개 출력
        "4  4": "ㄱ ㄱ", # ㄱ (첫번째 공백은 커밋만) (두번째 공백은 실제 공백) ㄱ
        "322 322" : "ㅠㅠ", # 모음만 2번 ' ㅠㅠ'
        "44 44" : "ㅋㅋ" ,
        "412  412": "가 가", # 가 (첫번째 공백 커밋) (두번째 공백 실제 공백) 가
        "412588": "갆", #ㄱ ㅏ ㄴ ㄴㅎ => 갆
        "41248": "갃", #ㄱ ㅏ ㄴ ㄱㅅ => 갆
        "412444": "갂", #ㄱ ㅏ ㄴ ㄱㄱ => 갆
        "41259": "갅", #ㄱ ㅏ ㄴ ㄴㅈ => 갆
        "412554": "갉", #ㄱ ㅏ ㄴ ㄹㄱ => 갆
        "4125500": "갊", #ㄱ ㅏ ㄴ ㄹㅁ => 갆
        "412557": "갋", #ㄱ ㅏ ㄴ ㄹㅂ => 갆
        "412558": "갌", #ㄱ ㅏ ㄴ ㄹㅅ => 갆
        "4125566": "갍", #ㄱ ㅏ ㄴ ㄹㅌ => 갆
        "4125577": "갎", #ㄱ ㅏ ㄴ ㄹㅍ => 갆
        "4125588": "갏", #ㄱ ㅏ ㄴ ㄹㅎ => 갆
        "01258801": '않이',
        "0221" : "여",
        "512535  442100773226621423088124732  92107230832  01751612" : "나는 컴퓨터공학부 정봉수 입니다", #'' + ''  = '' , 공백 두번이어야 실제 공백 인식 
        "4125126125512001271281201291299124412661277128812" : "가나다라마바사아자차카타파하", #간+ㅏ => 가나 디버깅
        "9921591015" : "천지인",
        "41258214 41  816121" : "간석기 시대", # 겹받침 lookahead 디버깅(간ㅅ + ㅓ+ㄱ)
        
        
    }

    print("--- Cheonjiin Automata Test ---")
    for seq, expected in test_cases.items():
        # if seq == "412 412": expected = "가가" # 수정된 공백 로직에 따른 예상 결과
        # if seq == " ": expected = ""
        # if seq == "  ": expected = " "
        # if seq == "   ": expected = "  "
        # if seq == "4  4": expected = "ㄱ ㄱ"
        # if seq == "412  412": expected = "가 가"


        result = automata.process_input_sequence(seq)
        print(f"Input: '{seq}', Expected: '{expected}', Got: '{result}', Match: {result == expected}")
        if result != expected :
            print("------------------------------------ MISMATCH DEBUG START")
            
            # print(f"Debug for Input: '{seq}'")
            for i, char_key_debug in enumerate(seq):
                print(f"--- Step {i+1}: Processing key: '{char_key_debug}' ---")
                automata.process_key(char_key_debug)
                print(f"Output Buffer: {''.join(automata.output_buffer)}")
                print(f"Cho: '{automata.cho}', Jung: '{automata.jung}', Jong: '{automata.jong}'")
                print(f"Vowel Input: '{automata.vowel_input_buffer}', Consonant Key: {automata.consonant_key_pressed}, Count: {automata.consonant_press_count}")
                print(f"Current State: {automata.current_state_enum}, LastWasSpace: {automata.last_key_was_space}")
            if automata.cho or automata.jung or automata.jong:
                # print(f"--- Finalizing remaining buffer before get_output ---")
                automata._commit_syllable()
                # print(f"Output Buffer after final commit: {''.join(automata.output_buffer)}")
            # final_debug_output = "".join(automata.output_buffer)
            final_debug_output = automata.get_output() # get_output()이 최종 결과를 반환하도록 함

            print(f"Final Debug State for '{seq}': State={automata.current_state_enum}, Output='{final_debug_output}'")
            print("------------------------------------ MISMATCH DEBUG END")

--- Cheonjiin Automata Test ---
Input: '4127', Expected: '갑', Got: '갑', Match: True
Input: '44120', Expected: '캉', Got: '캉', Match: True
Input: '412', Expected: '가', Got: '가', Match: True
Input: '8214', Expected: '석', Got: '석', Match: True
Input: '88823121', Expected: '쐐', Got: '쐐', Match: True
Input: '9315', Expected: '즨', Got: '즨', Match: True
Input: '41254', Expected: '간ㄱ', Got: '간ㄱ', Match: True
Input: '4125412', Expected: '간가', Got: '간가', Match: True
Input: '7771', Expected: '삐', Got: '삐', Match: True
Input: '0210', Expected: '엉', Got: '엉', Match: True
Input: '02188', Expected: '엏', Got: '엏', Match: True
Input: '412 412', Expected: '가가', Got: '가가', Match: True
Input: '0127 7124', Expected: '압박', Got: '압박', Match: True
Input: ' ', Expected: '', Got: '', Match: True
Input: '  ', Expected: ' ', Got: ' ', Match: True
Input: '   ', Expected: '  ', Got: '  ', Match: True
Input: '4  4', Expected: 'ㄱ ㄱ', Got: 'ㄱ ㄱ', Match: True
Input: '322 322', Expected: 'ㅠㅠ', Got: 'ㅠㅠ', Match: True
Inpu